## imports

In [1]:
from datetime import datetime
import re
import json

import pandas as pd
import numpy as np

from lib.data_viz_functions import *

In [2]:
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)

## reading data from disk

In [3]:
train_jane = pd.read_pickle("data/train_df_full_part1.pkl.zip", compression="zip")
train_baseline = pd.read_pickle(
    "data/all_auto_ru_09_09_2020.pkl.zip", compression="zip"
)
test = pd.read_pickle("data/test.pkl.zip", compression="zip")

train_jane.shape, train_baseline.shape, test.shape

((130201, 36), (89378, 26), (34686, 32))

## preprocessing functions

In [4]:
def parse_ownership_duration(train_str: str) -> float:

    """
    Returns ownership duration in days
    """

    baseline_str = "{'year': 2020, 'month': 9}"

    if not isinstance(train_str, str):
        return np.nan
    elif "year" in str(train_str):
        baseline_dict = json.loads(baseline_str.replace("'", '"'))
        train_dict = json.loads(train_str.replace("'", '"'))

        baseline_date = datetime.strptime(
            f"{baseline_dict['year']}-{baseline_dict['month']}-1", "%Y-%m-%d"
        )
        train_date = datetime.strptime(
            f"{train_dict['year']}-{train_dict['month']}-1", "%Y-%m-%d"
        )
        return (baseline_date - train_date).days
    elif " и " in str(train_str):
        return (
            int(train_str.split(" ")[0]) * 365
            + int(train_str.split(" и ")[1].split(" ")[0]) * 30
        )
    else:
        return int(train_str.split(" ")[0]) * 365


def get_number_of_owners_from_owners(in_str):
    if not isinstance(in_str, str):
        return None
    else:
        result = in_str.replace("\xa0", "")
        return int(re.sub("\D", "", result))
    # return string (to cat)


def get_engine_value(in_str):
    parsed_str = re.findall("(\d+.\d+)", in_str)
    if len(parsed_str):
        return float(parsed_str[0])
    else:
        return None


def get_bodytype(body_type):
    if isinstance(body_type, str):
        if "внедорожник 5 дв." in body_type:
            return "внедорожник 5 дв."
        elif "внедорожник 3 дв." in body_type:
            return "внедорожник 3 дв."
        elif "хэтчбек 5 дв." in body_type:
            return "хэтчбек 5 дв."
        elif "хэтчбек 3 дв." in body_type:
            return "хэтчбек 3 дв."
        elif "седан" in body_type:
            return "седан"
        elif "пикап двойная кабина" in body_type:
            return "пикап двойная кабина"
        elif "пикап полуторная кабина" in body_type:
            return "пикап полуторная кабина"
        elif "пикап одинарная кабина" in body_type:
            return "пикап одинарная кабина"
        elif "микровэн" in body_type:
            return "микровэн"
        elif "кабриолет" in body_type:
            return "кабриолет"
        elif "купе" in body_type:
            return "купе"
        elif "лифтбек" in body_type:
            "лифтбек"

        [
            "лифтбек",
            "внедорожник 5 дв.",
            "хэтчбек 5 дв.",
            "седан",
            "компактвэн",
            "универсал 5 дв.",
            "пикап одинарная кабина",
            "хэтчбек 3 дв.",
            "купе",
            "кабриолет",
            "минивэн",
            "пикап двойная кабина",
            "внедорожник 3 дв.",
            "родстер",
            "микровэн",
            "седан 2 дв.",
            "купе-хардтоп",
            "фастбек",
            "тарга",
            "внедорожник открытый",
            "лимузин",
            "пикап полуторная кабина",
            "седан-хардтоп",
            "фургон",
        ]


## quick view

In [5]:
describe_nums(train_jane.select_dtypes(exclude="object"))

,unique,n/a count,count,mean,std,min,25%,50%,75%,max
parsing_unixtime,130154,47,130154.000000,1632842487.144867,131636.029755,1632613481.000000,1632724602.500000,1632820776.000000,1632972687.500000,1633049056.000000
sell_id,40721,232,129969.000000,1104865960.875278,1779786.356124,1003139396.000000,1104900484.000000,1105218565.000000,1105376150.000000,1105402618.000000
mileage,15181,27167,103034.000000,116610.915407,85154.492046,1.000000,51000.000000,100000.000000,161000.000000,1000000.000000
price,8105,20436,109765.000000,2801223.260639,2689068.150068,23000.000000,1000000.000000,2045000.000000,3649000.000000,39990000.000000
views,4511,28203,101998.000000,564.982284,1814.036025,2.000000,127.000000,245.000000,511.000000,148149.000000
modelDate,53,215,129986.000000,2012.934578,6.017749,1938.000000,2010.000000,2014.000000,2017.000000,2021.000000
productionDate,50,213,129988.000000,2015.083854,5.929774,1952.000000,2012.000000,2016.000000,2020.000000,2021.000000
numberOfDoors,4,214,129987.000000,4.643164,0.619354,2.000000,4.000000,5.000000,5.000000,5.000000
model_info,0,130201,0.000000,nan,nan,nan,nan,nan,nan,nan
vendor,0,130201,0.000000,nan,nan,nan,nan,nan,nan,nan


In [6]:
describe_nums(train_baseline.select_dtypes(exclude="object"))

,unique,n/a count,count,mean,std,min,25%,50%,75%,max
mileage,17403,0,89378.000000,134828.811497,102979.784485,0.000000,56042.500000,130000.000000,194939.750000,1000000.000000
price,8168,410,88968.000000,1294586.356330,1950410.471999,16000.000000,350000.000000,650000.000000,1480000.000000,99000000.000000
enginePower,396,1,89377.000000,171.231637,83.527540,11.000000,115.000000,150.000000,199.000000,800.000000
productionDate,80,0,89378.000000,2010.832531,7.024776,1904.000000,2007.000000,2011.000000,2016.000000,2020.000000
modelDate,78,1,89377.000000,2008.543294,7.415436,1904.000000,2005.000000,2009.000000,2014.000000,2020.000000
numberOfDoors,5,1,89377.000000,4.510467,0.653203,0.000000,4.000000,5.000000,5.000000,5.000000
Владельцы,3,13588,75790.000000,2.233698,0.817455,1.000000,2.000000,2.000000,3.000000,3.000000
Таможня,1,0,nan,nan,nan,nan,nan,nan,nan,nan
Состояние,0,89378,0.000000,nan,nan,nan,nan,nan,nan,nan
hidden,0,89378,0.000000,nan,nan,nan,nan,nan,nan,nan


In [7]:
describe_nums(test.select_dtypes(exclude="object"))

,unique,n/a count,count,mean,std,min,25%,50%,75%,max
parsing_unixtime,34686,0,34686.000000,1603286733.619356,149307.058483,1603107306.000000,1603221157.750000,1603254133.000000,1603290080.250000,1603710264.000000
sell_id,34686,0,34686.000000,1098300150.880644,19112247.594511,2665.000000,1099048798.000000,1100910913.000000,1101245023.750000,1101374610.000000
mileage,11268,0,34686.000000,162009.767889,100676.559489,1.000000,91153.500000,149779.500000,215000.000000,1000000.000000
productionDate,69,0,34686.000000,2009.264602,7.047661,1904.000000,2006.000000,2011.000000,2014.000000,2020.000000
modelDate,66,0,34686.000000,2007.074728,7.415894,1904.000000,2004.000000,2008.000000,2012.000000,2020.000000
numberOfDoors,5,0,34686.000000,4.450816,0.703040,0.000000,4.000000,5.000000,5.000000,5.000000


In [8]:
train_jane.select_dtypes("object").shape, train_baseline.select_dtypes(
    "object"
).shape, test.select_dtypes("object").shape

((130201, 26), (89378, 16), (34686, 26))

In [9]:
train_jane.select_dtypes("object").describe().T.sort_values("unique", ascending=False)

,count,unique,top,freq
super_gen,130135,43825,"{'sale-data-attributes': {'asciiCat': 'cars', ...",235
image,130065,42933,https://avatars.mds.yandex.net/get-autoru-vos/...,241
car_url,129969,40721,https://auto.ru/cars/new/group/toyota/rav_4/21...,428
description,129988,35274,Официальный дилер Mercedes Benz ООО РОЛЬФ фили...,3230
equipment_dict,129969,31190,{},8214
complectation_dict,105267,2348,"['cruise-control', 'multi-wheel', 'airbag-pass...",1979
name,130153,2283,Nissan X-Trail III Рестайлинг,4013
region,103034,1335,в Москве,39180
date_added,103034,939,25 сентября,44873
vehicleConfiguration,129987,564,ALLROAD_5_DOORS AUTOMATIC 3.0,10895


In [10]:
train_baseline.select_dtypes("object").describe().T.sort_values(
    "unique", ascending=False
)

,count,unique,top,freq
description,86124,77568,РОЛЬФ Северо-Запад - это первый мегамолл автом...,246
start_date,89378,75680,2020-09-01T10:01:11Z,93
Комплектация,89378,5839,{'id': '0'},59130
name,89377,4484,2.0 AT (150 л.с.),1610
vehicleConfiguration,89377,1860,SEDAN MECHANICAL 1.6,4166
model,89378,1086,FOCUS,2467
engineDisplacement,89377,534,1.6,17032
Владение,29201,290,"{'year': 2018, 'month': 10}",497
bodyType,89377,168,Внедорожник 5 дв.,31885
brand,89378,36,MERCEDES,8012


In [11]:
train_jane["enginePower"] = train_jane["enginePower"].replace("undefined N12", None)
train_jane["enginePower"] = (
    train_jane[~pd.isna(train_jane["enginePower"])]["enginePower"]
    .str.split()
    .str.get(0)
    .astype("int")
)

In [12]:
train_jane["engineDisplacement"] = train_jane["engineDisplacement"].replace(
    " LTR", None
)
train_jane["engineDisplacement"] = (
    train_jane[~pd.isna(train_jane["engineDisplacement"])]["engineDisplacement"]
    .str.split()
    .str.get(0)
    .astype("float")
)

In [13]:
train_jane.engineDisplacement.head()

0    1.8
1    1.6
2    1.6
3    1.4
4    1.4
Name: engineDisplacement, dtype: float64

In [14]:
train_jane["used"] = train_jane["car_url"].str.contains("used")

In [15]:
train_jane[train_jane["used"] == False].dropna(thresh=24).shape

(0, 37)

In [16]:
test.iloc[34682]

bodyType                                                            седан
brand                                                                 BMW
car_url                 https://auto.ru/cars/used/sale/bmw/5er/1101369...
color                                                              чёрный
complectation_dict                                                    NaN
description                                Продаётся отличный автомобиль.
engineDisplacement                                                2.0 LTR
enginePower                                                       190 N12
equipment_dict                                                        NaN
fuelType                                                           дизель
image                   https://autoru.naydex.net/nwI1K7152/b604fdsYg7...
mileage                                                             98000
modelDate                                                            2016
model_info              {"code":"5ER",

In [17]:
train_jane["car_url"].str.contains("used").value_counts()

True     103034
False     26935
Name: car_url, dtype: int64

In [18]:
test.select_dtypes("object").describe().T.sort_values("unique", ascending=False)

,count,unique,top,freq
car_url,34686,34686,https://auto.ru/cars/used/sale/skoda/octavia/1...,1
image,34686,34557,https://avatars.mds.yandex.net/get-verba/21620...,13
description,34686,31732,Выгода до 82 000 руб. при обмене на Ваш автомо...,264
equipment_dict,24690,23705,"{""leather"":true}",108
super_gen,34686,5890,"{""id"":""6214876"",""displacement"":1598,""engine_ty...",193
name,34686,2780,1.6 AT (110 л.с.),631
complectation_dict,6418,2364,"{""id"":""4562904"",""name"":""Elegance"",""available_o...",51
model_info,34686,954,"{""code"":""OCTAVIA"",""name"":""Octavia"",""ru_name"":""...",1404
vehicleConfiguration,34686,634,ALLROAD_5_DOORS AUTOMATIC 3.0,2389
model_name,34686,544,OCTAVIA,1418


In [19]:
test["fuelType"].value_counts()

бензин     28601
дизель      5800
гибрид       223
электро       55
газ            7
Name: fuelType, dtype: int64

In [20]:
test.groupby("car_url")["image"].count().sort_values(ascending=False).head(10)

car_url
https://auto.ru/cars/used/sale/audi/100/1016944865-8eb8f/            1
https://auto.ru/cars/used/sale/skoda/octavia/1100434568-e9537421/    1
https://auto.ru/cars/used/sale/skoda/octavia/1100515744-d97a41fa/    1
https://auto.ru/cars/used/sale/skoda/octavia/1100514944-54916e94/    1
https://auto.ru/cars/used/sale/skoda/octavia/1100513036-35594e9f/    1
https://auto.ru/cars/used/sale/skoda/octavia/1100495414-bb724b6c/    1
https://auto.ru/cars/used/sale/skoda/octavia/1100492150-5ae5281f/    1
https://auto.ru/cars/used/sale/skoda/octavia/1100491772-7f29f9e8/    1
https://auto.ru/cars/used/sale/skoda/octavia/1100487934-c961ee31/    1
https://auto.ru/cars/used/sale/skoda/octavia/1100486714-6ff064ce/    1
Name: image, dtype: int64

In [21]:
test.iloc[2]["model_info"]

'{"code":"SUPERB","name":"Superb","ru_name":"Суперб","morphology":{},"nameplate":{"code":"","name":"","semantic_url":""}}'

In [22]:
test[test["vehicleConfiguration"] == "ALLROAD_5_DOORS AUTOMATIC 2.0"].sample(
    5, random_state=42
)

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,image,mileage,modelDate,model_info,model_name,name,numberOfDoors,parsing_unixtime,priceCurrency,productionDate,sell_id,super_gen,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня
23340,внедорожник 5 дв.,VOLKSWAGEN,https://auto.ru/cars/used/sale/volkswagen/tigu...,серебристый,NaN,Единственный владелец\nАвтомобиль с полным НДС...,2.0 LTR,140 N12,"{""cruise-control"":true,""asr"":true,""airbag-driv...",дизель,https://autoru.naydex.net/o9DBXQ270/5ac010hAY0...,70366,2011,"{""code"":""TIGUAN"",""name"":""Tiguan"",""ru_name"":""Ти...",TIGUAN,2.0d AT (140 л.с.) 4WD,5,1603213368,RUB,2015,1101291589,"{""id"":""7344497"",""displacement"":1968,""engine_ty...",ALLROAD_5_DOORS AUTOMATIC 2.0,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен
5517,внедорожник 5 дв.,HONDA,https://auto.ru/cars/used/sale/honda/cr_v/1101...,белый,NaN,✔У НАС КРЕДИТ ДАДУТ ВСЕМ!\n●ПРОСТО ОСТАВЬТЕ ЗА...,2.0 LTR,128 N12,"{""engine-proof"":true,""asr"":true,""adaptive-ligh...",бензин,https://autoru.naydex.net/mcQh17801/f1b5751dCi...,156000,1995,"{""code"":""CR_V"",""name"":""CR-V"",""ru_name"":""CR-V"",...",CR_V,2.0 AT (128 л.с.) 4WD,5,1603254705,RUB,1998,1101117481,"{""id"":""20501678"",""displacement"":1973,""engine_t...",ALLROAD_5_DOORS AUTOMATIC 2.0,автоматическая,JAPANESE,3 или более,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен
10155,внедорожник 5 дв.,BMW,https://auto.ru/cars/used/sale/bmw/x3/10960043...,красный,"{""id"":""2424078"",""name"":""xDrive20"",""available_o...","Продаю автомобиль БМВ-Х 3,2009 года выпуска,...",2.0 LTR,177 N12,"{""engine-proof"":true,""esp"":true,""airbag-driver...",дизель,https://autoru.naydex.net/lICN99c69/9bc7d3Rr7k...,158313,2006,"{""code"":""X3"",""name"":""X3"",""ru_name"":""Х3"",""morph...",X3,20d 2.0d AT (177 л.с.) 4WD,5,1603117968,RUB,2009,1096004334,"{""id"":""4917631"",""name"":""20d"",""nameplate"":""20d""...",ALLROAD_5_DOORS AUTOMATIC 2.0,автоматическая,EUROPEAN,3 или более,3 года и 5 месяцев,Оригинал,полный,Левый,Не требует ремонта,Растаможен
6177,внедорожник 5 дв.,HONDA,https://auto.ru/cars/used/sale/honda/cr_v/1100...,красный,NaN,"Авто в отличном состоянии для своих лет, ходов...",2.0 LTR,128 N12,NaN,бензин,https://avatars.mds.yandex.net/get-autoru-vos/...,319000,1995,"{""code"":""CR_V"",""name"":""CR-V"",""ru_name"":""CR-V"",...",CR_V,2.0 AT (128 л.с.) 4WD,5,1603257132,RUB,1998,1100628270,"{""id"":""20501678"",""displacement"":1973,""engine_t...",ALLROAD_5_DOORS AUTOMATIC 2.0,автоматическая,JAPANESE,3 или более,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен
4276,внедорожник 5 дв.,AUDI,https://auto.ru/cars/used/sale/audi/q5/1101303...,белый,NaN,Состояние нового автомобиля. Нет крашенных эле...,2.0 LTR,180 N12,"{""alloy-wheel-disks"":true,""ptf"":true,""aux"":tru...",бензин,https://avatars.mds.yandex.net/get-autoru-vos/...,31420,2012,"{""code"":""Q5"",""name"":""Q5"",""ru_name"":""Ку5"",""morp...",Q5,2.0 AT (180 л.с.) 4WD,5,1603129045,RUB,2016,1101303784,"{""id"":""20773921"",""displacement"":1984,""engine_t...",ALLROAD_5_DOORS AUTOMATIC 2.0,автоматическая,EUROPEAN,2 владельца,2 месяца,Оригинал,полный,Левый,Не требует ремонта,Растаможен


In [23]:
test["Владельцы"] = test["Владельцы"].apply(get_number_of_owners_from_owners)
train_jane["Владельцы"] = train_jane["Владельцы"].apply(
    get_number_of_owners_from_owners
)

In [24]:
print(
    "unique object cols in train (Jane's version):",
    set(train_jane.select_dtypes("object").columns.tolist())
    - set(test.select_dtypes("object").columns.tolist()),
    "\nunique object cols in test:",
    set(test.select_dtypes("object").columns.tolist())
    - set(train_jane.select_dtypes("object").columns.tolist()),
)

unique object cols in train (Jane's version): {'date_added', 'region', 'used'} 
unique object cols in test: {'enginePower', 'model_info', 'vendor', 'engineDisplacement'}


## making train and test similar

In [25]:
train_jane["model_name"] = train_jane.model_name.apply(lambda x: str(x).lower())
test["model_name"] = test.model_name.apply(lambda x: str(x).lower())

In [26]:
vendor_voc = (
    test[["brand", "vendor"]].drop_duplicates().set_index("brand").to_dict()["vendor"]
)
vendor_voc

{'SKODA': 'EUROPEAN',
 'AUDI': 'EUROPEAN',
 'HONDA': 'JAPANESE',
 'VOLVO': 'EUROPEAN',
 'BMW': 'EUROPEAN',
 'NISSAN': 'JAPANESE',
 'INFINITI': 'JAPANESE',
 'MERCEDES': 'EUROPEAN',
 'TOYOTA': 'JAPANESE',
 'LEXUS': 'JAPANESE',
 'VOLKSWAGEN': 'EUROPEAN',
 'MITSUBISHI': 'JAPANESE'}

In [27]:
train_jane.brand.unique().tolist()

['SKODA',
 nan,
 'AUDI',
 'HONDA',
 'VOLVO',
 'BMW',
 'NISSAN',
 'INFINITI',
 'MERCEDES',
 'TOYOTA',
 'LEXUS',
 'VOLKSWAGEN',
 'MITSUBISHI']

In [28]:
train_baseline.brand.unique().tolist()

['AUDI',
 'BMW',
 'CADILLAC',
 'CHERY',
 'CHEVROLET',
 'CHRYSLER',
 'CITROEN',
 'DAEWOO',
 'DODGE',
 'FORD',
 'GEELY',
 'HONDA',
 'HYUNDAI',
 'INFINITI',
 'JAGUAR',
 'JEEP',
 'KIA',
 'LEXUS',
 'MAZDA',
 'MINI',
 'MITSUBISHI',
 'NISSAN',
 'OPEL',
 'PEUGEOT',
 'PORSCHE',
 'RENAULT',
 'SKODA',
 'SUBARU',
 'SUZUKI',
 'TOYOTA',
 'VOLKSWAGEN',
 'VOLVO',
 'GREAT_WALL',
 'LAND_ROVER',
 'MERCEDES',
 'SSANG_YONG']

In [29]:
train_jane["vendor"] = train_jane["brand"].map(vendor_voc)
train_baseline["vendor"] = train_baseline["brand"].map(vendor_voc)
train_jane.vendor.unique().tolist(), train_baseline.vendor.unique().tolist()

(['EUROPEAN', nan, 'JAPANESE'], ['EUROPEAN', nan, 'JAPANESE'])

In [30]:
print(
    len(train_jane.loc[train_jane.vendor.isna()]["model_name"].unique().tolist()),
    "na of",
    len(train_jane.model_name.unique().tolist()),
)

1 na of 514


In [31]:
train_jane.loc[train_jane.vendor.isna()].shape

(213, 37)

In [32]:
train_jane.priceCurrency.unique()

array(['RUB', 'RUR', nan], dtype=object)

In [33]:
train_baseline.columns.sort_values().tolist()

['bodyType',
 'brand',
 'color',
 'description',
 'engineDisplacement',
 'enginePower',
 'fuelType',
 'hidden',
 'mileage',
 'model',
 'modelDate',
 'name',
 'numberOfDoors',
 'price',
 'productionDate',
 'start_date',
 'vehicleConfiguration',
 'vehicleTransmission',
 'vendor',
 'Владельцы',
 'Владение',
 'Комплектация',
 'ПТС',
 'Привод',
 'Руль',
 'Состояние',
 'Таможня']

## deleting cols which can't be used

at least for now - to re-review later

In [34]:
del test["car_url"]
del test["complectation_dict"]
del test["equipment_dict"]
del test["image"]
del test["model_info"]
del test["name"]
del test["parsing_unixtime"]
del test["priceCurrency"]
del test["sell_id"]
del test["vehicleConfiguration"]
del test["Состояние"]
del test["Таможня"]
del test["super_gen"]
del train_baseline["hidden"]
del train_baseline["name"]
del train_baseline["start_date"]
del train_baseline["vehicleConfiguration"]
del train_baseline["Комплектация"]
del train_baseline["Состояние"]
del train_baseline["Таможня"]
del train_jane["car_url"]
del train_jane["complectation_dict"]
del train_jane["date_added"]
del train_jane["equipment_dict"]
del train_jane["image"]
del train_jane["model_info"]
del train_jane["name"]
del train_jane["parsing_unixtime"]
del train_jane["priceCurrency"]
del train_jane["region"]
del train_jane["sell_id"]
del train_jane["vehicleConfiguration"]
del train_jane["views"]
del train_jane["Состояние"]
del train_jane["Таможня"]
del train_jane["super_gen"]
del train_jane["used"]

## view again

In [35]:
describe_nums(train_jane.select_dtypes(exclude="object"))

,unique,n/a count,count,mean,std,min,25%,50%,75%,max
mileage,15181,27167,103034.000000,116610.915407,85154.492046,1.000000,51000.000000,100000.000000,161000.000000,1000000.000000
price,8105,20436,109765.000000,2801223.260639,2689068.150068,23000.000000,1000000.000000,2045000.000000,3649000.000000,39990000.000000
enginePower,306,215,129986.000000,205.952133,97.503652,41.000000,146.000000,180.000000,249.000000,646.000000
modelDate,53,215,129986.000000,2012.934578,6.017749,1938.000000,2010.000000,2014.000000,2017.000000,2021.000000
engineDisplacement,51,629,129572.000000,2.437865,0.933884,0.700000,2.000000,2.000000,3.000000,6.600000
productionDate,50,213,129988.000000,2015.083854,5.929774,1952.000000,2012.000000,2016.000000,2020.000000,2021.000000
numberOfDoors,4,214,129987.000000,4.643164,0.619354,2.000000,4.000000,5.000000,5.000000,5.000000
Владельцы,3,27167,103034.000000,1.934264,0.847097,1.000000,1.000000,2.000000,3.000000,3.000000


In [36]:
describe_nums(train_baseline.select_dtypes(exclude="object"))

,unique,n/a count,count,mean,std,min,25%,50%,75%,max
mileage,17403,0,89378.000000,134828.811497,102979.784485,0.000000,56042.500000,130000.000000,194939.750000,1000000.000000
price,8168,410,88968.000000,1294586.356330,1950410.471999,16000.000000,350000.000000,650000.000000,1480000.000000,99000000.000000
enginePower,396,1,89377.000000,171.231637,83.527540,11.000000,115.000000,150.000000,199.000000,800.000000
productionDate,80,0,89378.000000,2010.832531,7.024776,1904.000000,2007.000000,2011.000000,2016.000000,2020.000000
modelDate,78,1,89377.000000,2008.543294,7.415436,1904.000000,2005.000000,2009.000000,2014.000000,2020.000000
numberOfDoors,5,1,89377.000000,4.510467,0.653203,0.000000,4.000000,5.000000,5.000000,5.000000
Владельцы,3,13588,75790.000000,2.233698,0.817455,1.000000,2.000000,2.000000,3.000000,3.000000


In [37]:
describe_nums(test.select_dtypes(exclude="object"))

,unique,n/a count,count,mean,std,min,25%,50%,75%,max
mileage,11268,0,34686.000000,162009.767889,100676.559489,1.000000,91153.500000,149779.500000,215000.000000,1000000.000000
productionDate,69,0,34686.000000,2009.264602,7.047661,1904.000000,2006.000000,2011.000000,2014.000000,2020.000000
modelDate,66,0,34686.000000,2007.074728,7.415894,1904.000000,2004.000000,2008.000000,2012.000000,2020.000000
numberOfDoors,5,0,34686.000000,4.450816,0.703040,0.000000,4.000000,5.000000,5.000000,5.000000
Владельцы,3,0,34686.000000,2.187338,0.835266,1.000000,1.000000,2.000000,3.000000,3.000000


In [38]:
train_jane.select_dtypes("object").shape, train_baseline.select_dtypes(
    "object"
).shape, test.select_dtypes("object").shape

((130201, 12), (89378, 13), (34686, 14))

In [39]:
train_jane.select_dtypes("object").describe().T.sort_values("unique", ascending=False)

,count,unique,top,freq
description,129988,35274,Официальный дилер Mercedes Benz ООО РОЛЬФ фили...,3230
model_name,130201,514,nan,27167
Владение,14286,261,3 года и 2 месяца,646
bodyType,129986,21,внедорожник 5 дв.,75068
color,129988,16,чёрный,42216
brand,129988,12,TOYOTA,20117
fuelType,129986,5,бензин,100407
vehicleTransmission,129986,4,автоматическая,73836
Привод,103032,3,полный,58563
vendor,129988,2,EUROPEAN,72959


In [40]:
train_baseline.select_dtypes("object").describe().T.sort_values(
    "unique", ascending=False
)

,count,unique,top,freq
description,86124,77568,РОЛЬФ Северо-Запад - это первый мегамолл автом...,246
model,89378,1086,FOCUS,2467
engineDisplacement,89377,534,1.6,17032
Владение,29201,290,"{'year': 2018, 'month': 10}",497
bodyType,89377,168,Внедорожник 5 дв.,31885
brand,89378,36,MERCEDES,8012
color,89378,16,040001,25212
fuelType,89378,6,бензин,75053
vehicleTransmission,89377,4,AUTOMATIC,49218
Привод,89377,3,передний,47315


In [41]:
test.select_dtypes("object").describe().T.sort_values("unique", ascending=False)[
    "top"
].head(1).tolist()

['Выгода до 82 000 руб. при обмене на Ваш автомобиль! \nВыгода при покупке в кредит до 100 000 руб.! \nРОЛЬФ ЯСЕНЕВО с пробегом. \n-Мы принимаем любые легковые и коммерческие автомобили независимо от марки, года выпуска, пробега и состояния автомобиля. \n-В наличии более 500 проверенных и готовых к продаже автомобилей. \n-Более 400 комплектов шин, дисков, колес в сборе. \n-Все автомобили проходят комплексную проверку - техническую, криминалистическую, проверку истории кузовных ремонтов сертифицированными специалистами. \n-Мы предоставляем гарантию юридической чистоты в соответствии с законодательством РФ. \n-Мы работаем каждый день с 9:00 до 22:00 (без выходных). \nУслуги: \n• Срочный выкуп Вашего автомобиля \n• Выкуп кредитных а/м \n• Trade-in - обмен Вашего автомобиля на новый или с пробегом (возможна доплата с любой стороны) \n• Выездная оценка Вашего а/м \n• Комиссионная продажа (на Ваших условиях) \n• Аукцион \n• Кредитование \n• Страхование КАСКО, ОСАГО \n• Дополнительное оборудо

In [42]:
test.sample(3).T

,16344,29845,29263
bodyType,седан,внедорожник 5 дв.,внедорожник 5 дв.
brand,MERCEDES,BMW,BMW
color,белый,золотистый,белый
description,Все вопросы по телефону. \nНедавно заменены це...,Зимняя резина в комплекте на оригинальных литы...,"В продаже автомобили от компании АО ABTODOM, о..."
engineDisplacement,1.8 LTR,2.0 LTR,2.0 LTR
enginePower,184 N12,150 N12,192 N12
fuelType,бензин,бензин,бензин
mileage,199000,165000,10500
modelDate,2011,2009,2015
model_name,c_klasse,x1,x1


In [43]:
train_jane.loc[train_jane.price.isna()].shape[0], train_jane.price.shape[
    0
], train_jane.loc[train_jane.price.isna()].shape[0] / train_jane.price.shape[0]

(20436, 130201, 0.1569573198362532)

## conclusion - to do about variables

$y = price$ - dropna, multiply for date course based coefficient for each dataset, take a log

- [ ] **description** - to tokenize - to read more about tokenize
- [ ] **engineDisplacement** - convert to float
- [ ] **enginePower** - convert to integer
- [ ] **Владельцы** - convert to integer
- [ ] **Владение** - calculate number of day
- [ ] **used** - is it possible to create this from urls or something for other datasets?
- [ ] **super_gen** - do we have something to extract? We have no such col in baseline
- [ ] **bodyType**, **color**, **brand**, **fuelType**, **vehicleTransmission**, **Привод**, **ПТС**, **Руль** - _temporary keep as is_

more

- [ ] mileage rename
- [ ] compare with existing features
- [ ] compare 4 dicts (equepment, complactation) train - test
Numerics - fill na, log if tailed, standartize
https://www.kaggle.com/datasets/gmbitz/all-auto-ru-09-09-2020

## more to do

check NA <= 5 () (new column)
make new or used column
electric - drop

make t-sne plot new-used - is it differ?

## checking datasets similarity

In [44]:
train_jane.shape, train_baseline.shape, test.shape

((130201, 20), (89378, 20), (34686, 19))

In [45]:
train_jane[train_jane.columns.sort_values().tolist()].sample(3, random_state=42).T

,83856,44922,77741
bodyType,внедорожник 5 дв.,внедорожник 5 дв.,внедорожник 5 дв.
brand,TOYOTA,NISSAN,TOYOTA
color,белый,чёрный,коричневый
description,Ваш выбор и Ваше время наши главные приоритеты...,Продажа от официального дилера ГК АВТОМИР Увел...,Продается легендарный Toyota RAV4 1 владелец П...
engineDisplacement,2.0,2.0,2.0
enginePower,148.0,141.0,158.0
fuelType,бензин,бензин,бензин
mileage,47800.0,184369.0,152712.0
modelDate,2016.0,2010.0,2010.0
model_name,c-hr,qashqai,rav4


In [46]:
train_baseline.rename(columns={"model": "model_name"}, inplace=True)
train_baseline[train_baseline.columns.sort_values().tolist()].sample(
    3, random_state=42
).T

,54772,73499,74260
bodyType,Компактвэн,Седан,Лифтбек
brand,OPEL,VOLKSWAGEN,VOLKSWAGEN
color,040001,FFD600,FAFBFB
description,Продам авто в хорошем состоянии без проблем вс...,"Штрафы оплаченны !! На днях снимут с машины, н...",Фольксваген Центр Подольск готов сделать вам п...
engineDisplacement,1.8,1.6,1.6
enginePower,140.0,90.0,90.0
fuelType,бензин,бензин,бензин
mileage,177000,380000,0
modelDate,2005.0,2014.0,2020.0
model_name,ZAFIRA,POLO,POLO


In [47]:
test[test.columns.sort_values().tolist()].sample(3, random_state=42).T

,19026,3337,11807
bodyType,внедорожник 5 дв.,седан,внедорожник 5 дв.
brand,TOYOTA,AUDI,NISSAN
color,чёрный,синий,белый
description,ЛОТ: 01215604\nФаворит Хофф\n\nВы можете получ...,Авто в идеальном состоянии проблем нет смотрит...,Комфорт: 6. Безопасность: 4. Обзор: 2. Салон: 2.
engineDisplacement,2.0 LTR,1.8 LTR,2.0 LTR
enginePower,150 N12,125 N12,141 N12
fuelType,бензин,бензин,бензин
mileage,121490,240000,104842
modelDate,2000,1994,2010
model_name,rav_4,a6,x_trail


## comments on further train_baseline processing

- [ ] bodyType - no doors specified
- [ ] color - hexified
- [ ] model_name - more standartified
- [ ] productionDate - different dtype
- [x] Владение - to convert to number of days
- [ ] vehicleTransmission - to standartize naming
- [ ] ПТС - to standartize naming
- [ ] Руль - to standartize naming

## merging train datasets

In [48]:
train_jane['sample'] = 1
train_baseline['sample'] = 2
train = pd.concat([train_jane, train_baseline])

In [49]:
train["Владение"] = train.Владение.apply(parse_ownership_duration)

In [50]:
train["Владение"].value_counts().head(5)

1155.0    646
2555.0    634
730.0     623
3650.0    603
365.0     525
Name: Владение, dtype: int64

## ------------------------

In [51]:
train.drop_duplicates(inplace=True)
train.dropna(subset=['price'], inplace = True)

In [52]:
train['bodyType'] = train['bodyType'].str.lower()

In [53]:
test['bodyType'].unique()

array(['лифтбек', 'внедорожник 5 дв.', 'хэтчбек 5 дв.', 'седан',
       'компактвэн', 'универсал 5 дв.', 'пикап одинарная кабина',
       'хэтчбек 3 дв.', 'купе', 'кабриолет', 'минивэн',
       'пикап двойная кабина', 'внедорожник 3 дв.', 'родстер', 'микровэн',
       'седан 2 дв.', 'купе-хардтоп', 'фастбек', 'тарга',
       'внедорожник открытый', 'лимузин', 'пикап полуторная кабина',
       'седан-хардтоп', 'фургон'], dtype=object)

In [54]:
train['bodyType'] = train['bodyType'].apply(get_bodytype)

In [55]:
train['bodyType']

0                     None
1                     None
2                     None
3                     None
4        внедорожник 5 дв.
               ...        
89373                 None
89374                 None
89375                 None
89376                 None
89377                 None
Name: bodyType, Length: 121420, dtype: object

In [56]:
train.shape

(121420, 21)

In [57]:
train[pd.isna(train['brand'])].count()

bodyType               0
brand                  0
color                  0
description            0
engineDisplacement     0
enginePower            0
fuelType               0
mileage                0
modelDate              0
model_name             0
numberOfDoors          0
productionDate         0
vehicleTransmission    0
vendor                 0
Владельцы              0
Владение               0
ПТС                    0
Привод                 0
Руль                   0
price                  0
sample                 0
dtype: int64